In [3]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from collections import defaultdict

# Load the locations data
locations = pd.read_csv('locations.txt', sep=' ', header=None, names=['id', 'city', 'lat', 'lon'])

# Initialize geocoder
geolocator = Nominatim(user_agent="city_mapper")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

# Dictionary to count occurrences of each city
city_counts = defaultdict(int)
city_names = []

# Reverse geocode each coordinate
for lat, lon in zip(locations['lat'], locations['lon']):
    try:
        location = reverse((lat, lon), exactly_one=True)
        if location and 'city' in location.raw['address']:
            city = location.raw['address']['city']
        elif location and 'town' in location.raw['address']:
            city = location.raw['address']['town']
        elif location and 'village' in location.raw['address']:
            city = location.raw['address']['village']
        else:
            city = "Unknown"
    except Exception as e:
        city = "Unknown"
    
    # Handle duplicate city names
    city_counts[city] += 1
    if city_counts[city] > 1:
        city_name = f"{city} ({city_counts[city]})"
    else:
        city_name = city
    city_names.append(city_name)

# Add the resolved city names to the DataFrame
locations['resolved_city'] = city_names

# Display the updated DataFrame
print(locations[['lat', 'lon', 'resolved_city']])

# Save city_names as a CSV file
pd.DataFrame(city_names, columns=['City Names']).to_csv('city_names.csv', index=False)


KeyboardInterrupt: 

In [4]:
import folium
import networkx
from folium.plugins import MarkerCluster
from networkx import center

center = [37.7749, -122.4194]

# Load city names from the CSV file
city_names_df = pd.read_csv('city_names.csv')

# Update the resolved_city column in the locations DataFrame
locations['resolved_city'] = city_names_df['City Names']

# Create a map centered on the mean coordinates
m = folium.Map(location=center, zoom_start=6)

# Add marker clustering for better visualization
marker_cluster = MarkerCluster().add_to(m)
for _, row in locations.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['resolved_city']  # Use the resolved_city column
    ).add_to(marker_cluster)

# Display the map
m


In [5]:
# Save the map to an HTML file
m.save("location_map.html")